In [4]:
import pandas as pd
import time
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
import csv
#import requests
#from http_request_randomizer.requests.proxy.requestProxy import RequestProxy
import threading
import math
import numpy as np
import os
from fake_useragent import UserAgent
import datetime
import shutil

In [5]:
def crawler(S,E,path):   
    
    ua = UserAgent()
    userAgent = ua.random
    
    option = webdriver.ChromeOptions()
    option.add_argument(f'user-agent={userAgent}')
    option.add_argument('--headless')
    option.add_argument("--incogniton")   
    option.add_argument('--no-sandbox')
    option.add_argument('--disable-gpu')
    option.add_argument('--disable-dev-shm-usage')
    try:
        driver = webdriver.Chrome(chrome_options=option,executable_path='/snap/bin/chromium.chromedriver')
    except: 
        return -2
            
    try:
        driver.get("https://twtmsearch.tipo.gov.tw/OS0/OS0103.jsp?l6=zh_TW&isReadBulletinen_US=&isReadBulletinzh_TW=true")
        time.sleep(2)


        #html = driver.page_source
        #print(html)

        #sdate & edate
        sdate = driver.find_element_by_name('q_appldateS')
        sdate.send_keys(S)

        edate = driver.find_element_by_name('q_appldateE')
        edate.send_keys(E)

        #11為最左上的輸入格
        element = driver.find_element_by_id('q_path11')
        element.send_keys(path)

        #不查詢其他應檢索
        check = driver.find_elements_by_name('caseItems')

        for i in check:
            if (i.get_attribute('value') == '4'):
                #time.sleep(3)
                driver.execute_script("arguments[0].click();",i)
                
        #查詢按鈕
        time.sleep(0.5)
        button = driver.find_element_by_id("search")
        button.click()
        time.sleep(2.5)

    except:
        driver.quit()
        return -2


    try:
            h2 = driver.find_element_by_css_selector('h2').text
    except:
            h2 = ""
            
            
   
    if h2 == "查無資料！":
        #無資料
        try:
            confirm = driver.find_element_by_class_name("confirm")
            confirm.click()
            driver.quit()
            return -3
        except:
            driver.quit()
            return -2

    else:
        try:
            #超過500確認按鈕
            cancel = driver.find_element_by_class_name("cancel")
            cancel.click()
            driver.quit()
            return -1

        except:
            try:
                #download excel
                down = driver.find_element_by_id("exportExcel")
                down.click()
                time.sleep(3)
            except:
                driver.quit()
                return -2
    
    driver.quit()
    return 1

def year(a):
    x = a.strftime("%Y/%m/%d")
    l = x.split("/")
    l[0] = str(int(l[0]) - 1911)
    x = l[0]+"/"+l[1]+"/"+l[2]
    return x

In [11]:
def readgroup():
    #Group List
    #os.chdir('/home/jack/4_Selenium')
    with open("top.txt",encoding="utf-8") as f:
        top = f.read().split("\n")
    with open("all.txt",encoding="utf-8") as f:
        allg = f.read().split("\n")
    #print(top,allg)
    
    #run = []
    run = {}
    for x in allg:
        #print(x)
        if x in top:
            head = x[:2]
            #print(head)
        else:
            #run.append((head+"-"+x[1]+"-00",x[3:]))
            run[head+"-"+x[1]+"-00"] = x[3:]
    # for i in run:
    #     print(i)
    #print(len(run))
    return run

In [12]:
#Temp for category csv
r = readgroup()
print(r)

{'01-A-00': '男人', '01-B-00': '女人', '01-C-00': '小孩/嬰兒', '01-D-00': '人物組合', '01-E-00': '性別不明', '01-F-00': '神怪', '01-G-00': '擬人化', '01-H-00': '人體部位', '02-A-00': '飛禽/蝙蝠', '02-B-00': '走獸', '02-C-00': '水族', '02-D-00': '昆蟲/蜘蛛', '02-E-00': '爬蟲/兩棲/其他', '02-F-00': '假想獸及古代動物', '02-G-00': '部份特徵', '02-H-00': '多種動物組合', '03-A-00': '樹', '03-B-00': '花/葉/草', '03-C-00': '根莖果蔬類', '03-D-00': '菇/蕈類', '03-E-00': '多種植物組合', '04-A-00': '天體', '04-B-00': '自然現象', '04-C-00': '地圖或模型', '04-D-00': '景觀', '05-A-00': '住屋', '05-B-00': '其他屋狀建築', '05-C-00': '非屋狀建築物', '05-D-00': '建築物之部份', '06-A-00': '機械用品(引擎.馬達.齒輪)', '06-B-00': '五金/園藝工具(釘.鎚.斧.剪.繩索.刀.馬蹄鐵.鐵鍊.水管)', '06-C-00': '度量設備(測時.測量.測長)如各種量尺', '06-D-00': '照明設備(手電筒.蠟燭.燈籠.火把.火燄.美術燈.油燈.燭台.電池)', '06-E-00': '計算工具(算盤.計算機.電腦.光碟)', '06-F-00': '通信設備(電話.天線.碟形天線.人造衛星.麥克風)', '06-G-00': '消防設備(滅火器.消防栓.雲梯)', '06-H-00': '傢俱(桌.椅.床.沙發.櫃子)', '06-I-00': '家用電器(電視.冰箱.吹風機.插頭.耳機)', '06-J-00': '衛浴設備(馬桶.浴缸.洗臉檯.水龍頭)', '06-K-00': '縫紉用品(針線.布尺.鈕釦.拉鍊.縫紉機)', '06-L-00': '吸煙用品(煙斗.香煙.煙灰缸.打火機)', '06-M-00': '

In [7]:
def runner(text):
    os.chdir('/home/jack/4_Selenium/processed')
    f = open("log.txt",'a+')
    f2 = open("sumlog.txt",'a+')
    f.write("====="+text+"=====\n")
    f2.write("====="+text+"=====\n")
    os.chdir('/home/jack/4_Selenium/temp')
    start = datetime.datetime(2010,1,1)
    TR = 180
    tr = TR

    srun = start
    erun = srun + datetime.timedelta(days = tr)
    end = datetime.datetime(2020,12,31) #change later
    #print(year(srun))
    #print(year(erun))

    
    
    #print(year(srun),year(erun),text)
    f.write(year(srun)+" "+year(erun)+" "+text+"\n")
    index = crawler(year(srun),year(erun),text)

    timecnt = 0
    succnt = 0
    failcnt = 0
    nodatacnt = 0

    while True:
        
        if timecnt > 30:
            TR = 100
        elif timecnt > 50:
            TR = 60

        if failcnt > 80:
            print("!!!Failed too much, next start from\n")
            print(year(srun),year(erun),text)
            f.write("!!!Failed too much, next start from\n")
            f.write(year(srun)+" "+year(erun)+" "+text+"\n")
            f2.write("!!!Failed too much, next start from\n")
            f2.write(year(srun)+" "+year(erun)+" "+text+"\n")
            break

        if index == -1:
            #print("time range failed\n")
            f.write("time range failed\n")
            timecnt += 1
            tr = tr - 30
            erun = srun + datetime.timedelta(days = tr)
            if erun > end:
                erun = end
            if erun < srun:
                erun = srun
            #print(year(srun),year(erun),text)
            f.write(year(srun)+" "+year(erun)+" "+text+"\n")
            index = crawler(year(srun),year(erun),text)
            
        elif index == -2:
            failcnt += 1
            #print("failed from other reason\n")
            f.write("failed from other reason\n")
            #print(year(srun),year(erun),text)
            f.write(year(srun)+" "+year(erun)+" "+text+"\n")
            index = crawler(year(srun),year(erun),text)

        elif index == -3:
            #print("no data\n")
            f.write("no data\n")
            if erun == end:
                print("~~~Done~~~")
                f.write("~~~Done~~~\n")
                f2.write("~~~Done~~~\n")
                break
            nodatacnt += 1
            tr = tr + 30
            erun = srun + datetime.timedelta(days = tr)
            if erun > end:
                erun = end
            #print(year(srun),year(erun),text)
            f.write(year(srun)+" "+year(erun)+" "+text+"\n")
            index = crawler(year(srun),year(erun),text)
            
        elif index == 1:
            dflag = False
            dirs = os.listdir()
            for a in dirs:
                if '.crdownload' in a:
                    #print("download failed\n")
                    f.write("download failed\n")
                    f.write("at %s :%s\n"%(text,a))
                    os.remove(a)
                    dflag = True
                    break

            if dflag:
                timecnt += 1
                tr = tr - 15
                erun = srun + datetime.timedelta(days = tr)
                if erun > end:
                    erun = end
                if erun < srun:
                    erun = srun
                #print(year(srun),year(erun),text)
                f.write(year(srun)+" "+year(erun)+" "+text+"\n")
                index = crawler(year(srun),year(erun),text)
            else:  
                #print("successed and continue\n")
                f.write("successed and continue\n")
                succnt += 1
                tr = TR
                if erun == end:
                    print("~~~Done~~~")
                    f.write("~~~Done~~~\n")
                    f2.write("~~~Done~~~\n")
                    break
                srun = erun + datetime.timedelta(days = 1)
                erun = srun + datetime.timedelta(days = tr)
                if erun > end:
                    erun = end
               
                #print(year(srun),year(erun),text)
                f.write(year(srun)+" "+year(erun)+" "+text+"\n")
                index = crawler(year(srun),year(erun),text)

        
    print("running data:",timecnt,succnt,failcnt,nodatacnt)
    f.write("running data: "+str(timecnt)+" "+str(succnt)+" "+str(failcnt)+" "+str(nodatacnt)+"\n")
    f2.write("running data: "+str(timecnt)+" "+str(succnt)+" "+str(failcnt)+" "+str(nodatacnt)+"\n")
    
    dirs = os.listdir()
    for x in dirs:
        shutil.move("/home/jack/4_Selenium/temp/%s"%x, "/service/trademark/labels/raw/%s/%s"%(text,x))
    
    f.write("=====done=====\n")
    f.close()
    f2.close()
    dirs = os.listdir()
    if len(dirs) == 0:
        return 1
    else:
        return 0

In [8]:
def excelhandle(text):
    os.chdir('/service/trademark/labels/raw/%s'%text)
    path = os.getcwd()
    #print(path)
    dirs = os.listdir()
    #print(dirs)
    #     for a in dirs:
    #         print(a)
    #     xls = pd.ExcelFile(r"%s"%a)
    #     sheet1 = xls.parse(0)
    #     casenos = [str(x) for x in sheet1['申請案號'].to_list()]
    #     fdates = [str(x) for x in sheet1['申請日/優先權日'].to_list()]
    #     names = [str(x) for x in sheet1['商標名稱'].to_list()]

    #     print(casenos[0])
    #     print(len(casenos))
    #     print(fdates[0])
    #     print(len(fdates))
    #     print(names[0])
    #     print(len(names))
    #     print("=====")

    with open('/home/jack/4_Selenium/processed/%s.csv'%text,'a+' ,newline='',encoding="utf-8") as fd:
        writer = csv.writer(fd)
        writer.writerow(['caseno','filing date','name'])
        for a in dirs:
            #print(a)
            xls = pd.ExcelFile(r"%s"%a)
            sheet1 = xls.parse(0)
            casenos = [str(x) for x in sheet1['申請案號'].to_list()]
            fdates = [str(x) for x in sheet1['申請日/優先權日'].to_list()]
            names = [str(x) for x in sheet1['商標名稱'].to_list()]
            xls.close()

            for i in range(len(casenos)):
                if casenos[i] == 'nan':
                    continue
                else:
                    writer.writerow([casenos[i],fdates[i],names[i]])
    shutil.copy("/home/jack/4_Selenium/processed/%s.csv"%text, "/service/trademark/labels/processed/%s.csv"%text)

In [7]:
# os.chdir('/service/trademark/labels/raw')
# print(os.getcwd())

# #make dir
# # for x in run:
# #     print(x)
# #     os.mkdir('%s'%x)
# fs = os.listdir()

# #delete all
# for x in fs:
#     os.chdir('/service/trademark/labels/raw/%s'%x)
#     if(os.listdir() != []):
#         for a in os.listdir():
#             os.remove(a)
#         print(x)
# os.chdir('/service/trademark/labels/processed')
# fs = os.listdir()
# for x in fs:
#     os.remove(x)
# os.chdir('/home/jack/4_Selenium/temp')
# fs = os.listdir()
# for x in fs:
#     os.remove(x)
# os.chdir('/home/jack/4_Selenium/processed')
# fs = os.listdir()
# for x in fs:
#     try:
#         os.remove(x)
#     except:
#         shutil.rmtree(x, ignore_errors=True)

/service/trademark/labels/raw


In [ ]:
#main
# cnt = 0
run = readgroup()
print(len(run))
for x in left:
    print("Start: %s"%x)
    r = runner(x)
    if r != 1:
        print("file not moved",x)
        os.chdir('/home/jack/4_Selenium/processed')
        with open("progress.txt",'a+') as f:
            f.write("not moved, terminate on: %s\n"%x)
        break
    excelhandle(x)
#     cnt += 1
#     if cnt == 10:
#         break
    os.chdir('/home/jack/4_Selenium/processed')
    with open("progress.txt",'a+') as f:
        f.write("done: %s\n"%x)

142
Start: 09-F-00


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:14: DeprecationWarning: use options instead of chrome_options
  


~~~Done~~~
running data: 0 23 0
Start: 05-A-00


In [20]:
failtest = ['09-D-00','12-A-00','09-E-00','03-B-00','11-A-00']
run = readgroup()
print(len(run))
left = list(set(run) - set(failtest))
print(len(left))

142
137


In [18]:
for a in failtest:
    excelhandle(a)

In [25]:
fail2 = ['11-B-00','13-I-00','08-E-00','06-B-00','07-C-00','12-G-00']

In [27]:
#Fail save
os.chdir('/service/trademark/labels/raw')
print(os.getcwd())

#make dir
# for x in run:
#     print(x)
#     os.mkdir('%s'%x)
fs = os.listdir()

#delete all
for x in fs:
    if x in fail2:
        os.chdir('/service/trademark/labels/raw/%s'%x)
        if(os.listdir() != []):
            for a in os.listdir():
                os.remove(a)
            print(x)
            
os.chdir('/service/trademark/labels/processed')
fs = os.listdir()
for x in fs:
    if x.split('.')[0] in fail2:
        print(x)
        os.remove(x)

os.chdir('/home/jack/4_Selenium/processed')
fs = os.listdir()
for x in fs:
     if x.split('.')[0] in fail2:
        print(x)
        os.remove(x)
        
        
#     try:
#         os.remove(x)
#     except:
#         shutil.rmtree(x, ignore_errors=True)

/service/trademark/labels/raw
13-I-00
12-G-00
11-B-00
11-B-00.csv
06-B-00.csv
08-E-00.csv
13-I-00.csv
12-G-00.csv
07-C-00.csv
13-I-00.csv
08-E-00.csv
11-B-00.csv
07-C-00.csv
12-G-00.csv
06-B-00.csv


In [28]:
for x in fail2:
    print("Start: %s"%x)
    r = runner(x)
    if r != 1:
        print("file not moved",x)
        os.chdir('/home/jack/4_Selenium/processed')
        with open("progress.txt",'a+') as f:
            f.write("not moved, terminate on: %s\n"%x)
        break
    excelhandle(x)
    os.chdir('/home/jack/4_Selenium/processed')
    with open("progress.txt",'a+') as f:
        f.write("done: %s\n"%x)

Start: 11-B-00


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:14: DeprecationWarning: use options instead of chrome_options
  


!!!Failed too much, next start from

106/10/16 106/10/26 11-B-00
running data: 718 235 81
Start: 13-I-00
~~~Done~~~
running data: 0 11 15
Start: 08-E-00
~~~Done~~~
running data: 0 23 0
Start: 06-B-00
~~~Done~~~
running data: 0 23 0
Start: 07-C-00
~~~Done~~~
running data: 0 23 0
Start: 12-G-00
~~~Done~~~
running data: 252 109 70


In [9]:
def runner_spe(text):
    os.chdir('/home/jack/4_Selenium/processed')
    f = open("log.txt",'a+')
    f2 = open("sumlog.txt",'a+')
    f.write("====="+text+"=====\n")
    f2.write("====="+text+"=====\n")
    os.chdir('/home/jack/4_Selenium/temp')
    start = datetime.datetime(2017,10,21)
    TR = 90
    tr = TR

    srun = start
    erun = srun + datetime.timedelta(days = tr)
    end = datetime.datetime(2020,12,31) #change later
    #print(year(srun))
    #print(year(erun))

    
    
    #print(year(srun),year(erun),text)
    f.write(year(srun)+" "+year(erun)+" "+text+"\n")
    index = crawler(year(srun),year(erun),text)

    timecnt = 0
    succnt = 0
    failcnt = 0
    nodatacnt = 0

    while True:
        
        if timecnt > 30:
            TR = 70
        elif timecnt > 50:
            TR = 50

        if failcnt > 200:
            print("!!!Failed too much, next start from\n")
            print(year(srun),year(erun),text)
            f.write("!!!Failed too much, next start from\n")
            f.write(year(srun)+" "+year(erun)+" "+text+"\n")
            f2.write("!!!Failed too much, next start from\n")
            f2.write(year(srun)+" "+year(erun)+" "+text+"\n")
            break

        if index == -1:
            #print("time range failed\n")
            f.write("time range failed\n")
            timecnt += 1
            tr = tr - 30
            erun = srun + datetime.timedelta(days = tr)
            if erun > end:
                erun = end
            if erun < srun:
                erun = srun
            #print(year(srun),year(erun),text)
            f.write(year(srun)+" "+year(erun)+" "+text+"\n")
            index = crawler(year(srun),year(erun),text)
            
        elif index == -2:
            failcnt += 1
            #print("failed from other reason\n")
            f.write("failed from other reason\n")
            #print(year(srun),year(erun),text)
            f.write(year(srun)+" "+year(erun)+" "+text+"\n")
            index = crawler(year(srun),year(erun),text)

        elif index == -3:
            #print("no data\n")
            f.write("no data\n")
            if erun == end:
                print("~~~Done~~~")
                f.write("~~~Done~~~\n")
                f2.write("~~~Done~~~\n")
                break
            nodatacnt += 1
            #tr = tr + 30
            srun = erun + datetime.timedelta(days = 1)
            erun = srun + datetime.timedelta(days = tr)
            if erun > end:
                erun = end
            #print(year(srun),year(erun),text)
            f.write(year(srun)+" "+year(erun)+" "+text+"\n")
            index = crawler(year(srun),year(erun),text)
            
        elif index == 1:
            dflag = False
            dirs = os.listdir()
            for a in dirs:
                if '.crdownload' in a:
                    #print("download failed\n")
                    f.write("download failed\n")
                    f.write("at %s :%s\n"%(text,a))
                    os.remove(a)
                    dflag = True
                    break

            if dflag:
                timecnt += 1
                tr = tr - 15
                erun = srun + datetime.timedelta(days = tr)
                if erun > end:
                    erun = end
                if erun < srun:
                    erun = srun
                #print(year(srun),year(erun),text)
                f.write(year(srun)+" "+year(erun)+" "+text+"\n")
                index = crawler(year(srun),year(erun),text)
            else:  
                #print("successed and continue\n")
                f.write("successed and continue\n")
                succnt += 1
                tr = TR
                if erun == end:
                    print("~~~Done~~~")
                    f.write("~~~Done~~~\n")
                    f2.write("~~~Done~~~\n")
                    break
                srun = erun + datetime.timedelta(days = 1)
                erun = srun + datetime.timedelta(days = tr)
                if erun > end:
                    erun = end
               
                #print(year(srun),year(erun),text)
                f.write(year(srun)+" "+year(erun)+" "+text+"\n")
                index = crawler(year(srun),year(erun),text)

        
    print("running data:",timecnt,succnt,failcnt,nodatacnt)
    f.write("running data: "+str(timecnt)+" "+str(succnt)+" "+str(failcnt)+" "+str(nodatacnt)+"\n")
    f2.write("running data: "+str(timecnt)+" "+str(succnt)+" "+str(failcnt)+" "+str(nodatacnt)+"\n")
    
    dirs = os.listdir()
    for x in dirs:
        shutil.move("/home/jack/4_Selenium/temp/%s"%x, "/service/trademark/labels/raw/%s/%s"%(text,x))
    
    f.write("=====done=====\n")
    f.close()
    f2.close()
    dirs = os.listdir()
    if len(dirs) == 0:
        return 1
    else:
        return 0

In [10]:
x = '11-B-00'
print("Start: %s"%x)
r = runner_spe(x)
if r != 1:
    print("file not moved",x)
    os.chdir('/home/jack/4_Selenium/processed')
    with open("progress.txt",'a+') as f:
        f.write("not moved, terminate on: %s\n"%x)
excelhandle(x)
os.chdir('/home/jack/4_Selenium/processed')
with open("progress.txt",'a+') as f:
    f.write("done: %s\n"%x)

Start: 11-B-00


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:14: DeprecationWarning: use options instead of chrome_options
  


~~~Done~~~
running data: 241 116 117 3
